# Router Query Engine with Sub-Question Query Engine and Pydantic MultiSelector

A custom router query engine capable of directing queries towards either a SQL database and/or vector database. It has the flexibility to choose the appropriate destination based on the specific requirements of the query

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import openai
import os
import config

#OpenAI API Key Authentication (The OpenAI API Key will be stored in the config.py file)
openai.api_key = config.openai_key

In [28]:
import pandas as pd

In [29]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SQLDatabase,
    WikipediaReader,
)

In [30]:
# Create Common Objects

from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
from llama_index.llms import OpenAI

# define node parser and LLM
chunk_size = 1024
llm = OpenAI(temperature=0, model="gpt-3.5-turbo", streaming=True)
service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
# text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size)


## Read data from CSV file

In [31]:
df = pd.read_csv("cars_revenue.csv")
df.head()

,ProductID,ProductName,CompanyName,SupplierID,CategoryID,UnitsSold,RevenueInDollars
0,1,Audi R8,Audi,1,1,299,134550000
1,2,Audi A7,Audi,1,2,4200,33600000
2,3,Audi A8,Audi,1,2,1593,175230000
3,4,Mercedes AMG ONE,Mercedes-Benz,2,1,275,748000000
4,5,Mercedes EQS,Mercedes-Benz,2,2,2403,300375000


In [32]:
df.dtypes

ProductID            int64
ProductName         object
CompanyName         object
SupplierID           int64
CategoryID           int64
UnitsSold            int64
RevenueInDollars     int64
dtype: object

## Create Database Schema

In [33]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [34]:

# Create an SQLAlchemy engine with an in-memory SQLite database
engine = create_engine("sqlite:///:memory:", future=True)

# Define a SQLAlchemy MetaData object
metadata_obj = MetaData()

# Define the columns for the table
table = Table(
    'products_car',  # Table name
    metadata_obj,
    Column('ProductID', Integer, primary_key=True),
    Column('ProductName', String),
    Column('CompanyName', String),
    Column('SupplierID', Integer),
    Column('CategoryID', Integer),
    Column('UnitsSold', Integer),
    Column('RevenueInDollars', Integer)
)

# Create the table in the database
metadata_obj.create_all(engine)

## Insert Data into Database Schema

In [35]:
#Insert data from dataframe into SQL table
df.to_sql('products_car', con=engine, if_exists='append', index=False)

5

In [36]:
# Checking whether the data is present
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM products_car")
    print(cursor.fetchall())

[(1, 'Audi R8', 'Audi', 1, 1, 299, 134550000), (2, 'Audi A7', 'Audi', 1, 2, 4200, 33600000), (3, 'Audi A8', 'Audi', 1, 2, 1593, 175230000), (4, 'Mercedes AMG ONE', 'Mercedes-Benz', 2, 1, 275, 748000000), (5, 'Mercedes EQS', 'Mercedes-Benz', 2, 2, 2403, 300375000)]


In [37]:
# print tables
metadata_obj.tables.keys()

dict_keys(['products_car'])

## Load data from Wikipedia

In [38]:
# Load Text Data
car_companies = ["Audi Company","Mercedes-Benz"]
wiki_docs = WikipediaReader().load_data(pages=car_companies)

## Build SQL Index

In [39]:
sql_database = SQLDatabase(engine, include_tables=["products_car"])


## Build Vector Index

In [40]:
# Insert documents into vector index
# Each document has metadata of the city attached

vector_indices = {}
vector_query_engines = {}

for car_company, wiki_doc in zip(car_companies, wiki_docs):
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    vector_indices[car_company] = vector_index
    vector_query_engines[car_company] = query_engine

## Define Query Engines and Set them as Tools

In [41]:
from llama_index.tools import ToolMetadata
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [42]:
#Define SQL Query Engine
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["products_car"],
)

In [43]:
#Define Vector Query Engines
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.query_engine import SubQuestionQueryEngine

query_engine_tools = []
for car_company in car_companies:
    query_engine = vector_query_engines[car_company]

    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=car_company, description=f"Provides information about {car_company}"
        ),
    )
    query_engine_tools.append(query_engine_tool)


#Define Subquestion Query Engine
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)


In [44]:
#Define Query Engine Tools
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "products_car, containing the units sold and revenue of each car"
    ),
)
s_engine_tool = QueryEngineTool.from_defaults(
    query_engine=s_engine,
    description=f"Useful for answering semantic questions about different car companies",
)

## Define Router Query Engine

In [51]:
# Define Router Query Engine
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector
from llama_index.selectors.pydantic_selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

query_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=([sql_tool] + [s_engine_tool]), service_context=service_context)

## Querying

In [60]:
response = query_engine.query(
    "What is the total Units Sold by Mercedes AMG ONE?"
)

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: The first choice mentions translating a natural language query into a SQL query, which could be useful for retrieving information about specific car models like Mercedes AMG ONE..
Selecting query engine 0: The first choice mentions translating a natural language query into a SQL query, which could be useful for retrieving information about specific car models like Mercedes AMG ONE..
Selecting query engine 0: The first choice mentions translating a natural language query into a SQL query, which could be useful for retrieving information about specific car models like Mercedes AMG ONE..
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR), CompanyName (VARCHAR), SupplierID (INTEGER), CategoryID (INTEGER), UnitsSold (INTEGER), RevenueInDollars (INTEGER) and foreign keys: .
> Table desc str: Table 'products_car' has columns: P

In [61]:
print(str(response))


 The total number of Mercedes AMG ONE units sold is 275.


In [64]:
response = query_engine.query(
    "Give a brief history about Audi and Mercedes-Benz"
)

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Can provide information about the revenue and units sold of Audi and Mercedes-Benz cars.
Selecting query engine 0: Can provide information about the revenue and units sold of Audi and Mercedes-Benz cars.
Selecting query engine 0: Can provide information about the revenue and units sold of Audi and Mercedes-Benz cars.
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR), CompanyName (VARCHAR), SupplierID (INTEGER), CategoryID (INTEGER), UnitsSold (INTEGER), RevenueInDollars (INTEGER) and foreign keys: .
> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR), CompanyName (VARCHAR), SupplierID (INTEGER), CategoryID (INTEGER), UnitsSold (INTEGER), RevenueInDollars (INTEGER) and foreign keys: .
> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR),

In [65]:
print(str(response))


Audi and Mercedes-Benz are two well-established car manufacturers with a rich history. Audi was founded in 1909 in Germany by August Horch. In 1932, it merged with three other companies to form Auto Union, and in the 1960s, Volkswagen acquired Auto Union, marking the beginning of the modern Audi era. Since then, Audi has become a global leader in luxury vehicles and has been known for its popular models like the Audi A8 and Audi R8. In recent years, Audi has also made significant investments in e-mobility and self-driving cars.

On the other hand, Mercedes-Benz has its origins in Karl Benz's invention of the first internal combustion engine in a car in 1886. The Benz Patent Motorwagen, financed by Bertha Benz's dowry, was the first automobile marketed by Daimler Motoren Gesellschaft (DMG) in 1901. In 1926, Karl Benz's company merged with Gottlieb Daimler's company to form Daimler-Benz, and the first Mercedes-Benz branded vehicles were produced. Mercedes-Benz gained popularity with its 

In [82]:
response = query_engine.query(
    "What is the revenue of Audi R8 and what technology is Audi famous for?"
)

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to retrieve the revenue of Audi R8 from the products_car table..
Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to retrieve the revenue of Audi R8 from the products_car table..
Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to retrieve the revenue of Audi R8 from the products_car table..
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR), CompanyName (VARCHAR), SupplierID (INTEGER), CategoryID (INTEGER), UnitsSold (INTEGER), RevenueInDollars (INTEGER) and foreign keys: .
> Table desc str: Table 'p

In [83]:
print(str(response))


The revenue of Audi R8 is $134,550,000. Audi is famous for its advanced technology, such as its Quattro all-wheel drive system, its turbocharged engines, and its advanced safety features.


In [87]:
response = query_engine.query(
    "Compare the revenue of Audi R8 and Mercedes AMG ONE"
)

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to compare the revenue of different cars..
Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to compare the revenue of different cars..
Selecting query engine 0: The first choice is relevant because it mentions translating a natural language query into a SQL query, which could be used to compare the revenue of different cars..
INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHAR), CompanyName (VARCHAR), SupplierID (INTEGER), CategoryID (INTEGER), UnitsSold (INTEGER), RevenueInDollars (INTEGER) and foreign keys: .
> Table desc str: Table 'products_car' has columns: ProductID (INTEGER), ProductName (VARCHA

In [88]:
print(str(response))

 The Mercedes AMG ONE has a significantly higher revenue than the Audi R8, with the AMG ONE bringing in $748 million compared to the R8's $134.5 million.
